In [7]:
from elasticsearch import Elasticsearch
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

from kafka import KafkaConsumer
import os
import json
from datetime import datetime

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

#consumer = KafkaConsumer('passenger', group_id = 1)
#for message in consumer:
#    message = json.loads(message.value)
#    print "{}".format(message)
#    incoming = passenger(message)
#    incoming.store()
#    consumer.commit()
#consumer.close()

In [9]:
driver_mapping = {
  'mappings': {
    'rolling': {
      'properties': {
        'id': {'type': 'string'},
        'status': {'type': 'string'},
        'location': {'type': 'geo_point', 'lat_lon': 'true'},
        'ctime': {'type': 'date'},
        'p1': {'type': 'string'},
        'p2': {'type': 'string'},
        'destination': {'type': 'geo_point', 'lat_lon': 'true'},
        'destinationid':{'type': 'string'},
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true'},
        'altdest2id':{'type': 'string'},                
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true'},
        'altdest2id':{'type': 'string'},                
      }
    }
  }
}


pass_mapping = {
  'mappings': {
    'rolling': {
      'properties': {
        'id': {'type': 'string'},
        'status': {'type': 'string'},
        'match': {'type': 'string'},                
        'location': {'type': 'geo_point', 'lat_lon': 'true'},
        'ctime': {'type': 'date'},
        'driver': {'type': 'string'},
        'destination': {'type': 'geo_point', 'lat_lon': 'true'},
        'destinationid':{'type': 'string'},                
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true'},
        'altdestid1':{'type': 'string'},                    
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true'},
        'altdestid2':{'type': 'string'},                
      }
    }
  }
}


#es.indices.delete(index='driver', ignore=[400, 404])
#es.indices.delete(index='passenger', ignore=[400, 404])
es.indices.create(index='driver', body=driver_mapping, ignore=400)
#es.indices.create(index='passenger', body=pass_mapping, ignore=400)

{u'acknowledged': True}

In [213]:
class driver(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='driver', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='driver', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())
        res = es.update(index='driver', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])
    def nearbyPassengers(self):
        geo_query = { "from" : 0, "size" : 3,
                      "_source":{"include": [ "_id" ]},
                      "query": {
                        "filtered": {
                           "query" : {
                                "term" : {"status": "wait"}
                            },
                            "filter": {
                                "geo_distance": {
                                  "distance":      "5km",
                                  "distance_type": "plane", 
                                  "location": self.location
                            }
                          }
                        }
                      }
                    }

        nearby = []
        res = es.search(index='passenger', doc_type='rolling', body=geo_query )
        for i in (res['hits']['hits']):
            nearby.append(i['_id'])
        return(nearby)
    
    def assignPassenger(self):
        if len(self.nearbyPassengers())>0: 
            p = getPassenger(self.nearbyPassengers()[0]) 
            p.status = 'pickup'
            self.status = 'pickup'
            self.destination = p.location
            self.destinationid = 'pickup_{}'.format(p.id)
            return True
        else:
            return False

    def loadPassenger(self, p):
        if self.p1 == None: 
            self.p1 = p.id
        elif self.p2 == None:
            self.p2 = p.id
        else:
            print('Cab is full')
            return False
        p.status = 'ontrip'
        self.status = 'ontrip'
        self.destination = p.destination
        self.destinationid = p.destinationid
        p.driver = self.id
        p.update()
        self.update()
        return True

    def updateLocation(self):
        if isinstance(self, 'driver'):
            obj_ = getDriver(self.id)
        elif isinstance(self, 'passenger'):
            obj_ = getPassenger(self.id)    
        else:
            return(False)
        obj_.location = self.location
        obj_.update()

    def arrived(self, p):
        if self.location == self.location:
            p.status == 'arrived'
        if self.p2 != None: 
            p2 = getPassenger(self.p2)
            p2.status == 'arrived'
        d.p1 == None
        d.p2 == None
        d.destination == None
        d.destinationid == None
        d.status == 'idle'


class passenger(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='passenger', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='passenger', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())        
        res = es.update(index='passenger', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])

    

def getPassenger(p_id):
    res = es.get(index='passenger', doc_type='rolling', id=p_id, ignore=404)
    return(passenger(res['_source'])) if res['found'] else res['found']

def getDriver(p_id):
    res = es.get(index='driver', doc_type='rolling', id=p_id, ignore=404)
    return(driver(res['_source'])) if res['found'] else res['found']
    
def sanityCheck(driver):
    if driver.isKnown():
        driverRecord = getDriverRecord(driver.id)
        return(driver.time > driverRecord.time)
    else:
        return(True)    
    
def getDriverRecord(id):
    res = es.get(index='trip', doc_type='driver', id=id)['_source']
    return(driver(res))

In [4]:
'''
    Driver:
    1. Check if timestamp make sense
    2. Check if driver exists in DB, if doesn't exists, create
    3. If cab's capacity is not zero, look for nearby requests
        1. If passenger request matched:
            1. Change status from idle to on trip (if necessary)
            2. Re-route / Set destination to passenger
            3. Save information to ElasticSearch
        2. If no nearby passenger:
            1. Update current location
    4. If cab's capacity is zero:
        Check if current location matches with destination:
        1. If matched, empty the cab and mark to idle
            1. Update number of trip
            2. Send trip info to kafka (for archive)
        2. If not matched, update current location
'''

"\n    Driver:\n    1. Check if timestamp make sense\n    2. Check if driver exists in DB, if doesn't exists, create\n    3. If cab's capacity is not zero, look for nearby requests\n        1. If passenger request matched:\n            1. Change status from idle to on trip (if necessary)\n            2. Re-route / Set destination to passenger\n            3. Save information to ElasticSearch\n        2. If no nearby passenger:\n            1. Update current location\n    4. If cab's capacity is zero:\n        Check if current location matches with destination:\n        1. If matched, empty the cab and mark to idle\n            1. Update number of trip\n            2. Send trip info to kafka (for archive)\n        2. If not matched, update current location\n"

In [214]:
if d.status in ['idle']:
    if not d.assignPassenger(): d.updateLocation()
    print(d.jsonFormat())
    print(p.jsonFormat())

elif d.status in ['pickup']:
    if d.location == d.destination:
        d.loadPassenger(d.destinationid)
    else:
        d.updateLocation()

elif d.status in ['ontrip']:
    if d.location == d.destination:
        d.arrived()
    else:
        d.assignPassenger()

{"status": "pickup", "p2": null, "p1": null, "altdest2id": null, "altdest2": null, "altdest1": null, "altdest1id": null, "destination": [40.74688644735875, -73.93986836685168], "name": "driver_1", "destinationid": "pickup_2", "location": [40.735281636440995, -73.93617159785171], "id": 1, "ctime": "2016-09-20T18:35:37.338147Z"}
{"status": "assigned", "altdest2id": "9/11 Memorial & Museum", "altdest2": [40.7114, -74.0125], "altdest1": [40.780829, -73.974004], "altdest1id": "American Museum of Natural History", "destination": [40.7831, -73.9592], "driver": null, "name": "passenger_10", "destinationid": "Solomon R. Guggenheim Museum", "location": [40.75573539924117, -73.93718173641139], "id": 10, "match": null, "ctime": "2016-09-20T18:59:29.212706Z"}


In [4]:
q = {
    "query" : { 
        "match_all" : {}
    }
}

In [6]:
es.delete_by_query(index='driver', body=q)

AttributeError: 'Elasticsearch' object has no attribute 'delete_by_query'

In [ ]:
def nearbyPassengers(self):
        geo_query = { "query" : { "match_all" : {} } }

        nearby = []
        res = es.search(index='passenger', doc_type='rolling', body=geo_query )
        for i in (res['hits']['hits']):
            nearby.append(i['_id'])
        return(nearby)

In [51]:
geo_query = { "fields": ['location','name'], "query" : { "match_all" : {} }}


In [52]:
res = es.search(index='passenger', doc_type='rolling', body=geo_query )['hits'].values()[0]
    


In [53]:
markers = []
for i in res:
    print i['fields']
    markers.append({'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png', 'infobox': i['fields']['name'], 'lat': i['fields']['location'][0], 'lng': i['fields']['location'][1]})

{u'name': [u'passenger_5'], u'location': [40.7904368518007, -73.949250506704]}
{u'name': [u'passenger_9'], u'location': [40.77427273567577, -73.94211122618668]}
{u'name': [u'passenger_10'], u'location': [40.75573539924117, -73.93718173641139]}
{u'name': [u'passenger_8'], u'location': [40.735687957659366, -73.98382792388858]}
{u'name': [u'passenger_2'], u'location': [40.74688644735875, -73.93986836685168]}
{u'name': [u'passenger_6'], u'location': [40.76655615538195, -73.97501206652214]}
{u'name': [u'passenger_4'], u'location': [40.78748930070537, -73.93026485004548]}
{u'name': [u'passenger_1'], u'location': [40.77668353316895, -73.95045330330294]}
{u'name': [u'passenger_7'], u'location': [40.753794479110105, -73.96141295175185]}
{u'name': [u'passenger_3'], u'location': [40.76460653914439, -73.99295512053497]}


In [50]:
markers

[{'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_5'],
  'lat': 40.7904368518007,
  'lng': -73.949250506704},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_9'],
  'lat': 40.77427273567577,
  'lng': -73.94211122618668},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_10'],
  'lat': 40.75573539924117,
  'lng': -73.93718173641139},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_8'],
  'lat': 40.735687957659366,
  'lng': -73.98382792388858},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_2'],
  'lat': 40.74688644735875,
  'lng': -73.93986836685168},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_6'],
  'lat': 40.76655615538195,
  'lng': -73.97501206652214},
 {'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png',
  'infobox': [u'passenger_4'],
  

In [55]:
        markers = []
        fullmap = Map(
                identifier="fullmap",
                varname="fullmap",
                style=(
                "height:100%;"
                "width:100%;"
                "top:0;"
                "left:0;"
                "position:absolute;"
                "z-index:200;"
                ),
                lat=40.7128,
                lng=-74.0059,
                geo_query = { "fields": ['location','name'], "query" : { "match_all" : {} }}
                res = es.search(index='passenger', doc_type='rolling', body=geo_query )['hits'].values()[0]
                for i in res:
                        markers.append({'icon': '//maps.google.com/mapfiles/ms/icons/blue-dot.png', 'infobox': i['fields']['name'], 'lat': i['fields']['location'][0], 'lng': i['fields']['location'][1]})
                ,
        # maptype = "TERRAIN",
        # zoom="5"
        )


SyntaxError: invalid syntax (<ipython-input-55-d0d3931e62b6>, line 16)